In [ ]:
from tqdm import tqdm
import pandas as pd 
from binance.client import Client
client = Client()

In [ ]:
from sqlalchemy import create_engine

In [ ]:
coins = ('BTCUSDT','ETHUSDT','BNBUSDT','SOLUSDT','ADAUSDT','XRPUSDT','DOTUSDT','LUNAUSDT',
        'DOGEUSDT','AVAXUSDT','SHIBUSDT','MATICUSDT','LTCUSDT','UNIUSDT','ALGOUSDT','TRXUSDT',
        'LINKUSDT','MANAUSDT','ATOMUSDT','VETUSDT')

In [ ]:
def getminutedata(symbol, lookback):
    frame = pd.DataFrame(client.get_historical_klines(symbol,
                                                     '1m',
                                                     lookback + ' days ago UTC'))
    frame = frame.iloc[:,:5]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close']
    frame[['Time', 'Open', 'High', 'Low', 'Close']] = frame[['Time', 'Open', 'High', 'Low', 'Close']].astype(float)
    frame.Time = pd.to_datetime(frame.Time, unit='ms')

    return frame

In [ ]:
engine = create_engine('sqlite:///CryptoTopCoin.db')

In [ ]:
for coin in tqdm(coins):
    getminutedata(coin, '30').to_sql(coin, engine, index=False)

In [ ]:
test = pd.read_sql('ETHUSDT', engine).set_index('Time')

In [ ]:
test

In [ ]:
import numpy as np

In [ ]:
def technicals(df):
    df = df.copy()
    df = df.resample('20min').ffill()
    df.dropna(inplace=True)
    df = df.copy()
    df['return'] = np.log(df.Close.pct_change() + 1)
    df['SMA_fast'] = df.Close.rolling(7).mean()
    df['SMA_slow'] = df.Close.rolling(25).mean()
    df['position'] = np.where(df['SMA_fast'] > df['SMA_slow'], 1, 0)
    df['strategyreturn'] = df['position'].shift(1) * df['return']
    df.dropna(inplace=True)
    return df

In [ ]:
technicals(test)

In [ ]:
np.exp(technicals(test)[['return', 'strategyreturn']].sum()) -1

In [ ]:
for coin in coins:
    df = pd.read_sql(coin, engine).set_index('Time')
    print(coin)
    trades = technicals(df).position.diff().value_counts().iloc[1:].sum()
    costs = trades * 0.00075
    print(np.exp(technicals(df)['return'].sum()) -1)
    print(np.exp(technicals(df)['strategyreturn'].sum()) -1 - costs)